In [26]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [27]:
# load the word index
word_index = imdb.get_word_index()

# reverse the word index to get words from indices
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [28]:
# load the pre-trained model
model = load_model('imdb_rnn_model.h5')

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [8]:
model.get_weights()  # to check the weights of the model

[array([[ 0.00724186, -0.12268964, -0.08626283, ...,  0.01204436,
         -0.05747947, -0.07726263],
        [ 0.04693659, -0.15032066, -0.02686106, ..., -0.0912238 ,
         -0.05869718, -0.05129045],
        [ 0.0282454 ,  0.04464116, -0.03296026, ..., -0.05349113,
         -0.00223088,  0.0146606 ],
        ...,
        [-0.06289903,  0.03102721,  0.08401418, ...,  0.0408077 ,
         -0.04627756,  0.11331426],
        [-0.02493133, -0.04643836,  0.02142882, ...,  0.02650878,
          0.02335212,  0.05903342],
        [-0.08206429,  0.03309351,  0.05952106, ...,  0.04084812,
          0.13901347,  0.06331932]], dtype=float32),
 array([[ 0.03993824, -0.14991395, -0.20228474, ...,  0.05535531,
         -0.11503243, -0.05274075],
        [ 0.02308341,  0.08644471,  0.01756303, ..., -0.05332104,
         -0.0917924 ,  0.08626275],
        [-0.04340224,  0.04497775, -0.26006493, ..., -0.00545241,
          0.01909512,  0.08361913],
        ...,
        [ 0.05164064,  0.12161523, -0.1

In [45]:
# helper function to preprocess the input text
# decode the reviews
def decode_review(encoded_reviews):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_reviews])

# preprocess the input review
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]  # 2 is the index for unknown words
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)    # pad to the same length as training data
    return padded_review

In [49]:
# prediction function
def predict_sentiment(review):
    # preprocess
    preprocessed_input = preprocess_text(review)

    prediction = model.predict(preprocessed_input)

    if prediction[0][0] > 0.05:
        sentiment = 'positive'
    else:
        sentiment = 'negative'
    
    return sentiment, prediction[0][0]

In [63]:
example_review = "The movie was very good, I really enjoyed it!"
sentiment, score = predict_sentiment(example_review)
print(f"Review: {example_review}\nSentiment: {sentiment}, Score: {score:4f}")
# score

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Review: The movie was very good, I really enjoyed it!
Sentiment: positive, Score: 0.095005
